In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Flatten,BatchNormalization,MaxPooling2D,Conv2D
from keras.models import Sequential,load_model

In [ ]:
train_data_dir = 'train'
validation_data_dir = 'validation'
num_classes = 6
img_rows,img_cols = 48,48
batch_size = 16
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.3,width_shift_range=0.3,
                                   height_shift_range=0.3,shear_range=0.3,rotation_range=30,
                                  vertical_flip=True,fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,target_size = (img_rows,img_cols),
                                                   color_mode= 'grayscale',batch_size=batch_size,class_mode = 'categorical',shuffle = True)
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,target_size=(img_rows,img_cols),
                                                             color_mode='grayscale',batch_size=batch_size,class_mode = 'categorical',shuffle = True)



In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'elu',kernel_initializer = 'he_normal',padding = 'same',input_shape = (img_rows,img_cols,1)))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation = 'elu',kernel_initializer = 'he_normal',padding = 'same',input_shape = (img_rows,img_cols,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes,activation='softmax',kernel_initializer='he_normal'))
print(model.summary())

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

                     
checkpoint = ModelCheckpoint("emotion3.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.4, patience = 5, verbose = 1, min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics = ['accuracy'])

nb_train_samples = 28273
nb_validation_samples = 3534
epochs = 10

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

In [1]:
from keras.models import load_model
classifier = load_model('emotion3.h5')

In [3]:
import cv2
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_alt.xml')
classifier = load_model('emotion3.h5')

class_lables={0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        if (np.sum([roi_gray])!=0):
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            #print("img_to_array",roi)
            roi = np.expand_dims(roi,axis = 0)
            #print("expand",roi)
            predict = classifier.predict(roi)[0]
            label = class_lables[predict.argmax()]

            cv2.putText(frame,label,(x,y),cv2.FONT_HERSHEY_SIMPLEX ,2,(0,255,123),2)
        else:
            cv2.putText(frame,'No Face Found',(10,50),cv2.FONT_HERSHEY_SIMPLEX ,2,(0,255,123),2)

    cv2.imshow('Emotion',frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()        
